# Introduction to Pandas

## Create DataFrame from file

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("data/C01-ncd_screen.csv")

In [ ]:
df[["hospcode", "pid"]]

## Exploring DataFrame

### Checking dimension

In [ ]:
hosp_info.shape

### Checking data type

In [ ]:
hosp_info.info()

#### Interpreting `object` column

In [ ]:
hosp_info["mname"][0]

In [ ]:
type(hosp_info["mname"][0])

### Read dataframe again with `hospcode` as string

In [ ]:
hosp_info = pd.read_csv("data/C00-hospital_info.csv", dtype={"hospcode":str})

### Viewing DataFrame

#### `.head()` and `.tail()`

In [ ]:
hosp_info.head()

In [ ]:
hosp_info.tail()

#### Sampling

In [ ]:
hosp_info.sample(n=5)

In [ ]:
hosp_info.sample(frac=0.0005)

### Sorting DataFrame

In [ ]:
hosp_info.sort_values(by="hospcode")

In [ ]:
hosp_info.sort_values(by=["mname", "dname"], ascending=[True, False])

### Selecing data

#### Select specific columns

In [ ]:
hosp_info["hospcode"]

In [ ]:
hosp_info[["hospcode", "hospname"]]

#### `.loc` and `.iloc`

In [ ]:
hosp_info.loc[[5, 19, 32], ["hospname", "address"]]

In [ ]:
hosp_info.iloc[[5, 19, 32], [3, 5]]

#### Conditional filtering

In [ ]:
hosp_info[hosp_info["dname"]=="กรมการแพทย์"]

In [ ]:
hosp_info[(hosp_info["dname"]=="กรมการแพทย์") &
          (hosp_info["haslatlog"]=="N")
         ] 

#### `.isin`

In [ ]:
hosp_info[hosp_info["dname"].isin(["กรมการแพทย์", "สำนักอนามัย"])]

### String method

In [ ]:
hosp_info[hosp_info["hospname"].str.contains("โรงพยาบาลส่งเสริมสุขภาพตำบล")]

# Challenge #1

ให้ใช้ข้อมูล C01-ncd_screen.csv เลือกข้อมูลที่ตรงกับเงื่อนไขต่อไปนี้
1. ผู้ที่มีความเสี่ยง Hypertension (SBP ≥ 140 หรือ DBP ≥ 90)
2. ผู้ที่มีความเสี่ยง Diabetes 
   - BSLEVEL ≥ 126 หรือ 
   - DMFAMILY=1 และ BSLEVEL ≥ 120 

In [ ]:
ncd_screen= pd.read_csv("data/C01-ncd_screen.csv")

In [ ]:

ncd_screen[(ncd_screen["sbp_1"] >= 140) | (ncd_screen["dbp_1"] >= 90) ]

In [ ]:

ncd_screen[ (ncd_screen["bslevel"] >= 126) | ((ncd_screen["dmfamily"] == 1) & (ncd_screen["bslevel"] >= 120))]

# Exploratory Data Analysis

In [ ]:
ncd_screen = pd.read_csv("data/C01-ncd_screen.csv", dtype={"hospcode":str,
                                                           "servplace":str,
                                                           "smoke":str,
                                                           "alcohol":str,
                                                           "dmfamily":str,
                                                           "htfamily":str,
                                                           "bstest":str,
                                                           "screenplace":str
                                                          })

## Basic statistics

### `.describe`

In [ ]:
ncd_screen.describe()

### Histogram

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ncd_screen["dbp_2"].plot.hist()
plt.show()

### Frequency: Countplot

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(data=ncd_screen, x="smoke")
plt.show()

# Challenge #2

ให้ใช้ข้อมูล C01-ncd_screen.csv และคำอธิบายข้อมูลจากไฟล์ 43files_structure.pdf หน้า 92 ทำการหาข้อผิดพลาดในข้อมูลในคอลัมน์ต่อไปนี้
- servplace
- alcohol
- dmfamily
- htfamily
- bstest

# Data Cleansing

## Missing data

In [ ]:
ncd_screen.isnull().any()

In [ ]:
ncd_screen.info()

### Select only data that has `bslevel` using `.dropna()`

In [ ]:
ncd_screen = ncd_screen.dropna(subset=["bslevel"])

In [ ]:
ncd_screen.isnull().any()

### Fill null value in `smoke`, `alcohol`, and `htfamily` with `9` using `.fillna()`

In [ ]:
ncd_screen["smoke"].fillna("9", inplace=True)
ncd_screen["alcohol"].fillna("9", inplace=True)
ncd_screen["htfamily"].fillna("9", inplace=True)

In [ ]:
ncd_screen.isnull().any()

## Invalid data

### Merge `0` in `smoke`, `alcohol`, and `htfamily` to `9` using `.replace`

In [ ]:
ncd_screen["smoke"].replace("0", "9", inplace=True)

In [ ]:
sns.countplot(data=ncd_screen, x="smoke")
plt.show()

## Outliers

### Check for outliers

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data=ncd_screen, x="weight")
plt.show()

In [ ]:
ncd_screen["weight"].quantile([0, 0.01, 0.05, 0.1,
                               0.25, 0.5, 0.75, 0.9,
                               0.95, 0.99, 0.999, 1
                              ])

### Remove rows with outliers

In [ ]:
ncd_screen = ncd_screen[(ncd_screen["weight"] <= 250) &
                        (ncd_screen["weight"] >= 1) &
                        (ncd_screen["height"] <= 220) &
                        (ncd_screen["height"] >= 1)
                       ]

In [ ]:
ncd_screen["weight"].plot.hist(bins=50)
plt.show()

# Challenge #4

ให้กำหนด correction logic ของคอลัมน์ต่อไปนี้ และทดลอง clean ข้อมูลด้วย logic ดังกล่าว
- sbp_1
- dbp_1
- sbp_2 **
- dbp_2 **
- bslevel

# Data Preparation

## Data enrichment / string extraction

### Calculation

In [ ]:
ncd_screen["bmi"] = ncd_screen["weight"] / (ncd_screen["height"]*0.01)**2

In [ ]:
ncd_screen[["hospcode", "pid", "weight", "height", "bmi"]]

### String extraction – split-and-extract

In [ ]:
hosp_info["address"].sample(5)

#### Test data

In [ ]:
test_data = "207 หมู่ 6 ต.พนมเศษ อ.ท่าตะโก จ.นครสวรรค์ 60160"

In [ ]:
test_data.split(" ")[-2].replace("จ.", "")

#### Define function

In [ ]:
def split_provice(address):
    return address.split(" ")[-2].replace("จ.", "")

#### Apply function

In [ ]:
hosp_info.dropna(subset=["address"], inplace=True)

In [ ]:
hosp_info["province"] = hosp_info["address"].apply(split_provice)

In [ ]:
hosp_info[["hospname", "address", "province"]].head()

### Merging

#### Join

In [ ]:
chospital = pd.read_csv("data/C02-chospital.csv", dtype=str)
service = pd.read_csv("data/C03-service.csv", dtype=str)

In [ ]:
service.merge(chospital, on="hospcode", how="inner")[["hospital_name"]+list(service.columns)]

## Summarization

### Descriptive statistics

In [ ]:
ncd_screen.describe()

### Simple aggregation

In [ ]:
ncd_screen.agg(Total_hospcode=("hospcode", "nunique"))

### Group aggregation

In [ ]:
ncd_screen.groupby("htfamily").agg(Total_patient=("pid", "nunique"),
                                   Mean_sbp1=("sbp_1", "mean"),
                                   Mean_dbp1=("dbp_1", "mean")
                                  )

## Reshaping data

### Long to wide - single row per unit

In [ ]:
data = pd.read_csv("data/C04-population_pyramid_2566.csv")

In [ ]:
data.pivot(index=["groupcode", "groupname"],
           columns="sex",
           values="count"
          )

### Long to wide - multiple rows per unit

In [ ]:
data = pd.read_csv("data/C05-diagnosis_opd.csv")

In [ ]:
data.pivot_table(index="hospcode",
                 columns="diagcode",
                 values="seq",
                 aggfunc="count",
                 fill_value=0
                ).sort_values(by="E041", ascending=False)

### Wide to long

In [ ]:
data = pd.read_csv("data/C06-population_by_age_sex_2566.csv", 
                   dtype={"hospcode":str, 
                          "areacode":str
                         }
                  )

In [ ]:
value_cols = [x for x in data.columns if "_g" in x]

In [ ]:
data.melt(id_vars=["hospcode", "areacode", "b_year"],
          value_vars=value_cols,
          value_name="count",
          var_name="group"
         )

# Saving DataFrame to File

In [ ]:
ncd_screen.to_csv("result.csv", index=None)

---